In [50]:
# !pip install beautifulsoup4
# !pip install geopy
!pip install geocoder

     |████████████████████████████████| 98 kB 5.6 MB/s eta 0:00:011


In [2]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

In [3]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
postalcodes_table=soup.find('table',{'class':"wikitable"})

In [229]:
df=pd.read_html(str(postalcodes_table))
# convert list to dataframe
df=pd.DataFrame(df[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [230]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df['Borough']!='Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [231]:
df.shape

(103, 3)

In [232]:
#get latitude and longitude from postal codes in Canada using pgeocode library
#!pip install pgeocode
import pgeocode
nomi = pgeocode.Nominatim('ca')

df['Latitude'] = df['Postal Code'].apply(lambda x: nomi.query_postal_code(x).latitude)
df['Longitude'] = df['Postal Code'].apply(lambda x: nomi.query_postal_code(x).longitude)
df.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [233]:
#reset index
df.reset_index(drop=True, inplace=True)

In [243]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
